In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy

In [2]:
# import the training dataset
train = pd.read_json('/content/drive/MyDrive/SQuAD/train-v2.0.json')
# validation dataset
valid = pd.read_json('/content/drive/MyDrive/SQuAD/dev-v2.0.json')
train.head(5)

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [3]:
train.iloc[2,1]['paragraphs'][0]

{'qas': [{'question': 'Who were Wang Jiawei and Nyima Gyaincain?',
   'id': '56cc239e6d243a140015eeb7',
   'answers': [{'text': 'Mainland Chinese scholars', 'answer_start': 274}],
   'is_impossible': False}],
 'context': "The exact nature of relations between Tibet and the Ming dynasty of China (1368–1644) is unclear. Analysis of the relationship is further complicated by modern political conflicts and the application of Westphalian sovereignty to a time when the concept did not exist. Some Mainland Chinese scholars, such as Wang Jiawei and Nyima Gyaincain, assert that the Ming dynasty had unquestioned sovereignty over Tibet, pointing to the Ming court's issuing of various titles to Tibetan leaders, Tibetans' full acceptance of these titles, and a renewal process for successors of these titles that involved traveling to the Ming capital. Scholars within China also argue that Tibet has been an integral part of China since the 13th century and that it was thus a part of the Ming Empire. 

In [4]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
text = train.iloc[2,1]['paragraphs'][0]['context']
nlp = spacy.load('en_core_web_md')
text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
#get rid of problem chars
text = ' '.join(text.split()) #a quick way of removing excess whitespace
doc = nlp(text)
sentences = []
for i in doc.sents:
  print(i)
  if len(i)>1:
    sentences.append(str(i).strip()) #tokenize into sentences
print(len(sentences))
print(sentences)

the exact nature of relations between tibet and the ming dynasty of china (1368–1644) is unclear.
analysis of the relationship is further complicated by modern political conflicts and the application of westphalian sovereignty to a time when the concept did not exist.
some mainland chinese scholars, such as wang jiawei and nyima gyaincain, assert that the ming dynasty had unquestioned sovereignty over tibet, pointing to the ming court's issuing of various titles to tibetan leaders, tibetans' full acceptance of these titles, and a renewal process for successors of these titles that involved traveling to the ming capital.
scholars within china also argue that tibet has been an integral part of china since the 13th century and that it was thus a part of the ming empire.
but most scholars outside china, such as turrell v. wylie, melvin c. goldstein, and helmut hoffman, say that the relationship was one of suzerainty, that ming titles were only nominal, that tibet remained an independent re

In [6]:
!pip install tensorflow-hub
import tensorflow_hub as hub
import tensorflow as tf

In [13]:
# 使用KerasLayer加载ELMo模型
elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", output_key="default",
                      trainable=False)

In [14]:
question = 'What is the states rank in size?'
q_list = [question]

# 在TensorFlow 2.x中，直接传递数据给elmo层
embeddings = elmo(tf.constant(q_list))

# 打印或处理embeddings变量
print(embeddings)

tf.Tensor(
[[ 0.07951442  0.05913033  0.10759461 ... -0.28353146  0.2741885
  -0.00219809]], shape=(1, 1024), dtype=float32)


In [15]:
from sklearn.metrics.pairwise import cosine_similarity
# 将句子转换为TensorFlow张量
sentences_tensor = tf.constant(sentences)

# 使用ELMo模型获取句子嵌入
embeddings = elmo(sentences_tensor)

# 计算余弦相似度
x_context = embeddings  # 假设x_context是你想要比较的嵌入向量集
x = embeddings  # 假设x是另一组嵌入向量，这里为了示例使用相同的嵌入

score = []
for index, i in enumerate(x_context):
    # 在TensorFlow 2.x中，i已经是一个嵌入向量，无需进一步处理
    statement = tf.expand_dims(i, axis=0)
    value = cosine_similarity(x, statement)[0][0]
    score.append((value, index))

score.sort()
e = score[-1][1]
ans = sentences[e]
print(ans)

the exact nature of relations between tibet and the ming dynasty of china (1368–1644) is unclear.


In [18]:
import nltk
from nltk import Tree
import spacy

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

# 使用完整的模型名称加载spaCy英语模型
en_nlp = spacy.load('en_core_web_md')

ques = "What is the states rank in size?"  # 示例句子或问题
[to_nltk_tree(sent.root).pretty_print() for sent in en_nlp(ques).sents]


      is                 
  ____|__________         
 |    |         rank     
 |    |     _____|____    
 |    |  states       in 
 |    |    |          |   
What  ?   the        size



[None]

In [19]:
[to_nltk_tree(sent.root).pretty_print()  for sent in
 en_nlp(ans).sents]

         is                                                     
    _____|________                                               
   |     |      nature                                          
   |     |    ____|____________                                  
   |     |   |    |            of                               
   |     |   |    |            |                                 
   |     |   |    |        relations                            
   |     |   |    |            |                                 
   |     |   |    |         between                             
   |     |   |    |            |                                 
   |     |   |    |          tibet                              
   |     |   |    |      ______|________                         
   |     |   |    |     |            dynasty                    
   |     |   |    |     |       ________|__________              
   |     |   |    |     |      |        |          of           
   |     |   |    

[None]